In [1]:
import numpy as np
import scipy.linalg

In [2]:
N = 3

u = 2 * np.random.randn(N) ** 2
v = 2 * np.random.randn(N) ** 2

A = np.random.randn(N, N)
A = 10 * np.eye(N) + A @ A.T

L = np.linalg.cholesky(A)

A_prime = A + np.outer(u, u) - np.outer(v, v)

In [3]:
A, L

(array([[11.4383179 ,  1.27114028, -0.21721925],
        [ 1.27114028, 14.35630702, -3.41958074],
        [-0.21721925, -3.41958074, 13.32731246]]),
 array([[ 3.38205823,  0.        ,  0.        ],
        [ 0.37584814,  3.7702845 ,  0.        ],
        [-0.06422694, -0.90057956,  3.53725088]]))

In [4]:
A_prime, np.linalg.cholesky(A_prime)

(array([[15.83538485, 10.01719734, -0.09369509],
        [10.01719734, 31.7199122 , -3.28967961],
        [-0.09369509, -3.28967961, 12.9229821 ]]),
 array([[ 3.97936991,  0.        ,  0.        ],
        [ 2.51728228,  5.03817448,  0.        ],
        [-0.02354521, -0.64118654,  3.53713267]]))

In [5]:
D = np.diag(L).copy()
L = L / D
D = D ** 2

In [6]:
L, D

(array([[ 1.        ,  0.        ,  0.        ],
        [ 0.11113   ,  1.        ,  0.        ],
        [-0.01899049, -0.2388625 ,  1.        ]]),
 array([11.4383179 , 14.2150452 , 12.51214381]))

In [7]:
L @ np.diag(D) @ L.T

array([[11.4383179 ,  1.27114028, -0.21721925],
       [ 1.27114028, 14.35630702, -3.41958074],
       [-0.21721925, -3.41958074, 13.32731246]])

In [8]:
Z = np.hstack((u[:, None], v[:, None]))
B = np.diag([1, -1])

Z, B

(array([[2.11845284, 0.30128973],
        [4.23979688, 0.78247835],
        [0.15126677, 0.65361457]]),
 array([[ 1,  0],
        [ 0, -1]]))

In [9]:
H = scipy.linalg.solve_triangular(L, Z, lower=True)
V = H / D[:, None]

In [10]:
X = V.T @ V
Lambda, Q = np.linalg.eigh(X)
X_sqrt = Q @ np.diag(np.sqrt(Lambda)) @ Q.T
X_rsqrt = Q @ np.diag(1.0 / np.sqrt(Lambda)) @ Q.T

IXB = np.eye(2) + X_sqrt @ B @ X_sqrt
Lambda_IXB, Q_IXB = np.linalg.eigh(IXB)
IXB_sqrt = Q_IXB @ np.diag(np.sqrt(Lambda_IXB)) @ Q_IXB.T

C = X_rsqrt @ (-np.eye(2) + IXB_sqrt) @ X_rsqrt
C

array([[ 0.48557026,  0.00306146],
       [ 0.00306146, -0.50095951]])

In [16]:
(np.eye(N) + V @ C @ V.T)

array([[1.01633809, 0.02469081, 0.00741254],
       [0.02469081, 1.03723236, 0.01085426],
       [0.00741254, 0.01085426, 1.00187679]])

In [11]:
(np.eye(N) + V @ B @ V.T), (np.eye(N) + V @ C @ V.T) @ (np.eye(N) + V @ C @ V.T)

(array([[1.0336077 , 0.05078477, 0.01522809],
        [0.05078477, 1.07657842, 0.02231604],
        [0.01522809, 0.02231604, 1.00392985]]),
 array([[1.0336077 , 0.05078477, 0.01522809],
        [0.05078477, 1.07657842, 0.02231604],
        [0.01522809, 0.02231604, 1.00392985]]))

In [17]:
C_s = C
W = Z
W_bar_s = ((H.T / D[None, :]) @ H)

L_prime = np.eye(N) + V @ C @ V.T

for s in range(N):
    f_hat = C_s @ H[s, :]
    theta = 1 + H[s, :] @ f_hat / D[s]
    W_bar_s = W_bar_s - np.outer(H[s, :], H[s, :]) / D[s]
    y = W_bar_s @ f_hat
    k = C @ y
    x_bar = f_hat.T @ y / D[s]
    g = np.sqrt(theta ** 2 + x_bar)
    r_s = ((1 + theta) * f_hat + k) / (g ** 2 * D[s])
    q = -((1 + theta + g) * f_hat + k) / (g * D[s] * (theta + g))
    C_s = C_s + np.outer(q, f_hat)
    d_bar_s = g ** 2 * D[s]
    
    for t in range(s + 1, N):
        w_t_s = V[t, :] - L_prime[t, s] * H[s, :].T
        L_prime[t, s] = w_t_s @ r_s

In [18]:
L_prime

array([[1.01633809, 0.02469081, 0.00741254],
       [0.02134402, 1.03723236, 0.01085426],
       [0.02722256, 0.02928731, 1.00187679]])